# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [11]:
file_path = "../WeatherPy/cities.csv"
cities_weather_df = pd.read_csv(file_path)
# cities_weather_df
del cities_weather_df["Unnamed: 0"]
cities_weather_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,mataura,100.0,NZ,1.620709e+09,89.0,-46.1927,168.8643,46.00,11.01
1,onega,100.0,RU,1.620709e+09,89.0,63.9061,38.1404,49.15,8.63
2,hasaki,75.0,JP,1.620709e+09,59.0,35.7333,140.8333,66.00,6.91
3,bluff,100.0,NZ,1.620709e+09,89.0,-46.6000,168.3333,46.00,11.01
4,salta,90.0,AR,1.620709e+09,100.0,-24.7859,-65.4117,51.80,3.44
...,...,...,...,...,...,...,...,...,...
561,peachtree city,90.0,US,1.620710e+09,100.0,33.3968,-84.5958,64.99,3.71
562,safakulevo,4.0,RU,1.620710e+09,30.0,54.9901,62.5442,62.80,4.38
563,wattegama,40.0,LK,1.620710e+09,74.0,6.7989,81.4808,87.80,4.61
564,buenaventura,87.0,CO,1.620710e+09,79.0,3.8801,-77.0312,64.00,2.01


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [12]:
gmaps.configure(g_key)

In [13]:
locations = cities_weather_df[["Lat", "Lng"]]
humidity = cities_weather_df["Humidity"].astype(float)
max_humidity = max(humidity)

In [14]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max_humidity, 
                                 point_radius=3.5)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [15]:
# Perfect temperature (Between 70 and 80 Degrees)
perfect_conditions = cities_weather_df[(cities_weather_df["Max Temp"] >70) 
                                       & (cities_weather_df["Max Temp"] < 80) 
                                       & (cities_weather_df["Wind Speed"] < 10) 
                                       & (cities_weather_df["Cloudiness"] == 0)]

perfect_conditions

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
5,alhandra,0.0,BR,1.620709e+09,94.0,-7.4386,-34.9144,73.40,5.75
168,saint-philippe,0.0,RE,1.620709e+09,47.0,-21.3585,55.7679,77.00,5.75
190,oussouye,0.0,SN,1.620709e+09,88.0,12.4850,-16.5469,71.60,4.61
220,saint-joseph,0.0,RE,1.620709e+09,47.0,-21.3667,55.6167,77.00,5.75
238,pahrump,0.0,US,1.620709e+09,9.0,36.2083,-115.9839,72.00,1.01
297,fasa,0.0,IR,1.620709e+09,33.0,28.9383,53.6482,73.40,2.46
401,najran,0.0,SA,1.620709e+09,21.0,17.4924,44.1277,78.80,3.74
422,nicoya,0.0,CR,1.620709e+09,78.0,10.1483,-85.4520,77.00,5.75
428,gat,0.0,IL,1.620709e+09,100.0,31.6100,34.7642,71.01,1.63
464,cabedelo,0.0,BR,1.620710e+09,100.0,-6.9811,-34.8339,71.60,4.61


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [16]:
hotel_df = pd.DataFrame(perfect_conditions)
hotel_df["Hotel Name"] = ""
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
5,alhandra,0.0,BR,1.620709e+09,94.0,-7.4386,-34.9144,73.40,5.75,
168,saint-philippe,0.0,RE,1.620709e+09,47.0,-21.3585,55.7679,77.00,5.75,
190,oussouye,0.0,SN,1.620709e+09,88.0,12.4850,-16.5469,71.60,4.61,
220,saint-joseph,0.0,RE,1.620709e+09,47.0,-21.3667,55.6167,77.00,5.75,
238,pahrump,0.0,US,1.620709e+09,9.0,36.2083,-115.9839,72.00,1.01,
297,fasa,0.0,IR,1.620709e+09,33.0,28.9383,53.6482,73.40,2.46,
401,najran,0.0,SA,1.620709e+09,21.0,17.4924,44.1277,78.80,3.74,
422,nicoya,0.0,CR,1.620709e+09,78.0,10.1483,-85.4520,77.00,5.75,
428,gat,0.0,IL,1.620709e+09,100.0,31.6100,34.7642,71.01,1.63,
464,cabedelo,0.0,BR,1.620710e+09,100.0,-6.9811,-34.8339,71.60,4.61,


In [19]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
